# 나랑 닮은 연예인은 누구?
1. 얼굴임베딩 만들기(1) 얼굴인식

In [8]:
import face_recognition
import os
import matplotlib.pyplot as plt
import matplotlib.image as img
import numpy as np
from PIL import Image

#얼굴 자르는 함수
def get_cropped_face(image_path):
    # [[YOUR CODE]]
    image = face_recognition.load_image_file(image_path)
    face_locations = face_recognition.face_locations(image)

    print(face_locations)  # 이미지에서 얼굴 영역의 좌표를 출력합니다.

    if face_locations:
        a, b, c, d = face_locations[0]
        cropped_face = image[a:c,d:b,:]
        return cropped_face
    else:
        return []
    
    
    return cropped_face

- images 디렉토리 안에 있는 모든 이미지 파일의 임베딩을 구해서 dict 구조에 담아 리턴하는 함수

In [19]:
# 얼굴 임베딩 모델을 이용하여 우리가 다운로드해 둔 6장의 이미지에서 얼굴 임베딩을 추출
dir_path = os.getenv('HOME')+'/aiffel/photo/'
file_list = os.listdir(dir_path)

print ("file_list: {}".format(file_list))

#images 디렉토리 안에 있는 모든 이미지 파일의 임베딩을 구해서 dict 구조에 담아 리턴하는 함수

def get_face_embedding(face):
    return face_recognition.face_encodings(face)

# 얼굴 임베딩 벡터를 구하는 함수는 face_recognition.face_encodings()
# images 디렉토리 안에 있는 모든 이미지 파일의 임베딩을 구해서 dict 구조에 담아 리턴하는 함수
def get_face_embedding_dict(dir_path):
    file_list = os.listdir(dir_path)
    embedding_dict = {}
    
    for file in file_list:
        img_path = os.path.join(dir_path, file)
        face = get_cropped_face(img_path)
        print(img_path)
        
        embedding = get_face_embedding(face)
        if len(embedding) > 0:  
        # 얼굴영역 face가 제대로 detect되지 않으면  len(embedding)==0인 경우가 발생하므로 
        # os.path.splitext(file)[0]에는 이미지파일명에서 확장자를 제거한 이름이 담깁니다. 
            embedding_dict[os.path.splitext(file)[0]] = embedding[0]
        
    return embedding_dict

file_list: ['7.jpeg', '6.jpeg', '5.jpeg', '0.jpeg', '9.jpeg', '8.jpeg', '4.jpeg', '3.jpeg', '.ipynb_checkpoints', '2.jpeg']


In [20]:
embedding_dict = get_face_embedding_dict(dir_path)
#embedding_dict['0']

[(266, 489, 489, 266)]
/aiffel/aiffel/photo/7.jpeg
[(68, 163, 175, 55)]
/aiffel/aiffel/photo/6.jpeg
[(68, 235, 175, 127)]
/aiffel/aiffel/photo/5.jpeg
[(1313, 1490, 2112, 691)]
/aiffel/aiffel/photo/0.jpeg
[(112, 349, 379, 82)]
/aiffel/aiffel/photo/9.jpeg
[(98, 339, 253, 184)]
/aiffel/aiffel/photo/8.jpeg
[(260, 617, 528, 349)]
/aiffel/aiffel/photo/4.jpeg
[(80, 151, 187, 44)]
/aiffel/aiffel/photo/3.jpeg


IsADirectoryError: [Errno 21] Is a directory: '/aiffel/aiffel/photo/.ipynb_checkpoints'

- get_distance() 함수는 인자로 파일명의 사람 이름 2개를 받아서, 두 사람의 얼굴 임베딩 사이의 거리(L2 distance)를 구하는 형태

In [16]:
def get_distance(name1, name2):
    return np.linalg.norm(embedding_dict[name1]-embedding_dict[name2], ord=2)

get_distance('0', '1')

NameError: name 'embedding_dict' is not defined

이전 스텝에서 우리는 다음 기능을 함수로 구현해 보았습니다.

def get_cropped_face(image_file) : 이미지 파일에서 얼굴 영역을 가져오는 함수
def get_face_embedding(face) : 얼굴영역으로부터 얼굴 임베딩 벡터를 구하는 함수
def get_face_embedding_dict(dir_path) : 디렉토리 안에 있는 모든 이미지의 임베딩 딕셔너리를 구하는 함수
def get_distance(name1, name2) : 두 이미지(사람 이름) 사이의 임베딩 벡터 거리를 구하는 함수

특정 이미지와 가장 닮은 이미지를 구해보는 기능을 만들 수 있을 것

In [17]:
def get_nearest_face(name, top=5):
    sort_key_func = get_sort_key_func(name)
    sorted_faces = sorted(embedding_dict.items(), key=lambda x:sort_key_func(x[0]))
    
    for i in range(top+1):
        if i == 0 :   # 첫번째로 나오는 이름은 자기 자신일 것이므로 제외합시다. 
            continue
        if sorted_faces[i]:
            print('순위 {} : 이름({}), 거리({})'.format(i, sorted_faces[i][0], sort_key_func(sorted_faces[i][0])))

In [18]:
# 나와 가장 닮은 사람은 누굴까요?
get_nearest_face('0')

NameError: name 'get_sort_key_func' is not defined

## 회고

이번 프로젝트에 첫번째 문제는 임베딩 과정에서 입력받는 매개변수 문제로 동일한 사진만 반복해서 임베딩 된 경우가 있다.

두번째로는 외부에서 받아온 사진을 임베딩 하는 과정에서 반복적으로 오류가 발생하여 끝내 해결하지 못했다.

이번 프로젝트를 마치면서 아쉬운 점은 첫번째 문제의 경우 혼자서 많은(최소 3시간이상) 시간을 혼자서 고민했는데, 게더에 다른 분에게 여쭤보고 10분만에 바로 해결했다. 이런 것에 시간과 멘탈을 낭비한게 아쉽다.

다음 프로젝트에서 어려운 것이 있으면 적당히 고민한 후 물어보고 해결해야겠다.